

The new openinsider data is scraped into a csv

The cell 'Initial cleaning' from creating training data set ipynb is run on that data to load in the csv and clean.

Ticker prices are downloaded for each symbol, prev 100 days and the following 35 or something 

Ticker prices are put into the new trades using the 'Putting 90days of prices into df_by_ticker' cell

prev day of SPY history is downloaded.

The create_features() def is run to create all the new features.

In [1]:
import yfinance as yf 
import pandas as pd
import numpy as np
from datetime import time, timedelta, datetime
from pandas.tseries.offsets import BMonthEnd
from ta.momentum import RSIIndicator
from ta.trend import MACD




In [2]:
#import sys
#del sys.modules['features']


In [3]:
import openinsider_scraper as OIScraper
import features as ft


c:\Users\azzag\Documents\Programming\Python\TradingBot\features.py:21: SyntaxWarning: "\$" is an invalid escape sequence. Such sequences will not work in the future. Did you mean "\\$"? A raw string is also an option.
  .str.replace('[\$,\+\-\%>]', '', regex=True)


In [4]:
scraper = OIScraper.OpenInsiderScraper()
startdate="2025-07-01" # Will always round down to the first of the month

df = scraper.scrape(startdate)


2025-10-21 14:38:37,204 - openinsider - INFO - Starting scraping process...
Processing months: 100%|██████████| 4/4 [00:00<00:00, 35320.45it/s]
2025-10-21 14:38:37,214 - openinsider - INFO - Scraping completed. Found 2585 transactions.
2025-10-21 14:38:37,224 - openinsider - INFO - Data saved to data\insider_trades.csv


In [5]:
df = ft.clean_scraped_data(df)


In [6]:
df = ft.download_ticker_prices(df)


c:\Users\azzag\Documents\Programming\Python\TradingBot\features.py:73: FutureWarning: YF.download() has changed argument auto_adjust default to True
  downloaded_prices = yf.download(
[**************        30%                       ]  263 of 878 completedHTTP Error 404: {"quoteSummary":{"result":null,"error":{"code":"Not Found","description":"Quote not found for symbol: PPTIN"}}}
[**********************53%                       ]  467 of 878 completedHTTP Error 404: {"quoteSummary":{"result":null,"error":{"code":"Not Found","description":"Quote not found for symbol: BLSK"}}}
[*********************100%***********************]  878 of 878 completed

9 Failed downloads:
['NRX']: YFPricesMissingError('possibly delisted; no price data found  (1d 2015-10-21 00:00:00 -> 2025-12-19 19:20:43)')
['PPTIN', 'BLSK', 'EMBY', 'UHALB', 'DMII', 'PHXE.', 'EMIS', 'LKSP']: YFTzMissingError('possibly delisted; no timezone found')


In [7]:
# 

df = ft.add_features(df)
df = df.dropna()
df = df[(df[['last_price', 'Qty', 'shares_held', 'Owned', 'Value', 'Owned_norm', 'title_rank', 'filing_lad_days', 'same_day_trade_count']] >= 0).all(axis=1)]

Starting create features


c:\Users\azzag\Documents\Programming\Python\TradingBot\features.py:130: RuntimeWarning: invalid value encountered in scalar divide
  (ticker_data['Volume'].iloc[-1] - ticker_data['Volume'].mean())
c:\Users\azzag\Documents\Programming\Python\TradingBot\features.py:130: RuntimeWarning: invalid value encountered in scalar divide
  (ticker_data['Volume'].iloc[-1] - ticker_data['Volume'].mean())
c:\Users\azzag\Documents\Programming\Python\TradingBot\features.py:130: RuntimeWarning: invalid value encountered in scalar divide
  (ticker_data['Volume'].iloc[-1] - ticker_data['Volume'].mean())


Finish create features
Starting count_same_day_trades
Finished count_same_day_trades


In [8]:
df = df[df['transaction_date'] < pd.to_datetime('2025-9-15')].copy()


### test update latest data def

In [9]:
df = ft.update_latest_data(df)

2025-10-21 14:39:15,286 - openinsider - INFO - Starting scraping process...
2025-10-21 14:39:15,286 - openinsider - INFO - Starting scraping process...


Scraping data


Processing months: 100%|██████████| 2/2 [00:00<00:00, 24174.66it/s]
2025-10-21 14:39:15,293 - openinsider - INFO - Scraping completed. Found 916 transactions.
2025-10-21 14:39:15,293 - openinsider - INFO - Scraping completed. Found 916 transactions.
2025-10-21 14:39:15,298 - openinsider - INFO - Data saved to data\insider_trades.csv
2025-10-21 14:39:15,298 - openinsider - INFO - Data saved to data\insider_trades.csv


Cleaning data


c:\Users\azzag\Documents\Programming\Python\TradingBot\features.py:73: FutureWarning: YF.download() has changed argument auto_adjust default to True
  downloaded_prices = yf.download(
[*********************100%***********************]  390 of 390 completed

8 Failed downloads:
['PPTIN', 'BLSK', 'EMBY', 'UHALB', 'DMII', 'PHXE.', 'EMIS', 'LKSP']: YFTzMissingError('possibly delisted; no timezone found')


Adding features, this may take a while.
Starting create features


c:\Users\azzag\Documents\Programming\Python\TradingBot\features.py:130: RuntimeWarning: invalid value encountered in scalar divide
  (ticker_data['Volume'].iloc[-1] - ticker_data['Volume'].mean())
c:\Users\azzag\Documents\Programming\Python\TradingBot\features.py:130: RuntimeWarning: invalid value encountered in scalar divide
  (ticker_data['Volume'].iloc[-1] - ticker_data['Volume'].mean())
c:\Users\azzag\Documents\Programming\Python\TradingBot\features.py:130: RuntimeWarning: invalid value encountered in scalar divide
  (ticker_data['Volume'].iloc[-1] - ticker_data['Volume'].mean())


Finish create features
Starting count_same_day_trades
Finished count_same_day_trades
Removeing trades with out of range or missing values.


## Update df
### Takes a df with data up to a certain date, and adds data from that date to today.

input:

df

filtered = True

In [10]:
'''
date_newest_trade = df['transaction_date'].max() #datetine of most recent trade.
startdate = str(date_newest_trade.date())
scraper = OIScraper.OpenInsiderScraper()
#startdate="2025-07-01" # Will always round down to the first of the month

print('Scraping data')
df_new = scraper.scrape(startdate)
print('Cleaning data')
df_new = ft.clean_scraped_data(df_new)
print('Downloading ticker prices')
df_new = ft.download_ticker_prices(df_new)
print('Adding features, this may take a while.')
df_new = ft.add_features(df_new)
df_new = df_new[df_new['transaction_date'] > date_newest_trade] #scrape() rounds down to the first of the month, this ensures only new trades are merged.

if filtered:
    print('Removeing trades with out of range or missing values.')
    df_new = df_new.dropna()
    df_new = df_new[(df_new[['last_price', 'Qty', 'shares_held', 'Owned', 'Value', 'Owned_norm', 'title_rank', 'filing_lad_days', 'same_day_trade_count']] >= 0).all(axis=1)]

merged_df = pd.concat([df, df_new], ignore_index=True)

#return merged_df
'''

'\ndate_newest_trade = df[\'transaction_date\'].max() #datetine of most recent trade.\nstartdate = str(date_newest_trade.date())\nscraper = OIScraper.OpenInsiderScraper()\n#startdate="2025-07-01" # Will always round down to the first of the month\n\nprint(\'Scraping data\')\ndf_new = scraper.scrape(startdate)\nprint(\'Cleaning data\')\ndf_new = ft.clean_scraped_data(df_new)\nprint(\'Downloading ticker prices\')\ndf_new = ft.download_ticker_prices(df_new)\nprint(\'Adding features, this may take a while.\')\ndf_new = ft.add_features(df_new)\ndf_new = df_new[df_new[\'transaction_date\'] > date_newest_trade] #scrape() rounds down to the first of the month, this ensures only new trades are merged.\n\nif filtered:\n    print(\'Removeing trades with out of range or missing values.\')\n    df_new = df_new.dropna()\n    df_new = df_new[(df_new[[\'last_price\', \'Qty\', \'shares_held\', \'Owned\', \'Value\', \'Owned_norm\', \'title_rank\', \'filing_lad_days\', \'same_day_trade_count\']] >= 0).al

Delete later

In [11]:
import joblib


In [12]:
df = joblib.load('saved_vars/All_features_and_prices_df.joblib')


In [13]:
df = ft.update_latest_data(df, filtered=False)


2025-10-21 20:55:44,413 - openinsider - INFO - Starting scraping process...
2025-10-21 20:55:44,413 - openinsider - INFO - Starting scraping process...
2025-10-21 20:55:44,413 - openinsider - INFO - Starting scraping process...


Scraping data


Processing months: 100%|██████████| 3/3 [00:00<00:00, 28468.13it/s]
2025-10-21 20:55:44,423 - openinsider - INFO - Scraping completed. Found 2113 transactions.
2025-10-21 20:55:44,423 - openinsider - INFO - Scraping completed. Found 2113 transactions.
2025-10-21 20:55:44,423 - openinsider - INFO - Scraping completed. Found 2113 transactions.
2025-10-21 20:55:44,432 - openinsider - INFO - Data saved to data\insider_trades.csv
2025-10-21 20:55:44,432 - openinsider - INFO - Data saved to data\insider_trades.csv
2025-10-21 20:55:44,432 - openinsider - INFO - Data saved to data\insider_trades.csv


Cleaning data


c:\Users\azzag\Documents\Programming\Python\TradingBot\features.py:73: FutureWarning: YF.download() has changed argument auto_adjust default to True
  downloaded_prices = yf.download(
[*********************100%***********************]  786 of 786 completed

8 Failed downloads:
['PPTIN', 'BLSK', 'EMBY', 'UHALB', 'DMII', 'PHXE.', 'EMIS', 'LKSP']: YFTzMissingError('possibly delisted; no timezone found')


Adding features, this may take a while.
Starting create features


c:\Users\azzag\Documents\Programming\Python\TradingBot\features.py:130: RuntimeWarning: invalid value encountered in scalar divide
  (ticker_data['Volume'].iloc[-1] - ticker_data['Volume'].mean())
c:\Users\azzag\Documents\Programming\Python\TradingBot\features.py:130: RuntimeWarning: invalid value encountered in scalar divide
  (ticker_data['Volume'].iloc[-1] - ticker_data['Volume'].mean())
c:\Users\azzag\Documents\Programming\Python\TradingBot\features.py:130: RuntimeWarning: invalid value encountered in scalar divide
  (ticker_data['Volume'].iloc[-1] - ticker_data['Volume'].mean())


Finish create features
Starting count_same_day_trades
Finished count_same_day_trades


In [15]:
joblib.dump(df, 'saved_vars/35_features_2015_21-10-2025.joblib')


['saved_vars/35_features_2015_21-10-2025.joblib']